In [1]:
import pandas as pd

Пусть имеются данные о займах (loans), выданных различными организациями (originators). Уникальный идентификатор займа - loanId, уникальный идентификатор организации - originatorId. Из параметров займа известны сумма - amount, срок - term, комиссии - fees, дата выдачи в формате UNIXTIMESTAMP - issuedDatetime, дата погашения в формате UNIXTIMESTAMP - repaidDatetime. Организации находятся в разных странах, и сумма займа отображается в валюте конкретной организации. Таблица originators содержит соотношения между организациями originators и валютами currency. Для каждой организации может быть только одна валюта.

In [164]:
loans = pd.read_csv('data/loans.csv')
loans.head()

,originatorId,loanId,amount,term,rate,fees,issuedDatetime,repaidDatetime
0,2,1492427,200.0,20,NaN,61.0,1438276988,1.440170e+09
1,2,1360769,700.0,30,NaN,0.0,1432733814,1.440168e+09
2,2,1453132,800.0,30,NaN,0.0,1437469946,1.440146e+09
3,2,1514642,500.0,10,NaN,99.0,1439402251,1.440141e+09
4,2,1537517,500.0,30,NaN,0.0,1440499423,1.441029e+09


In [44]:
originators = pd.read_csv('data/originators.csv')
originators.head()

,originatorId,currency
0,1,1
1,2,7
2,6,3
3,8,7
4,16,8


Также имеются исторические данные о курсах валют в таблице rates. Представлено значение курса одной валюты (sourceCurrencyId) к другой валюте (targetCurrencyId) в конкретный день day. Предполагается, что в течение дня курс валют не изменялся.

In [119]:
rates = pd.read_csv('data/currency_rates.csv')
rates.head()

,day,sourceCurrencyId,targetCurrencyId,rate
0,20151008,1,5,0.020877
1,20151008,1,4,0.010473
2,20151008,1,7,0.060175
3,20151008,1,2,0.016002
4,20151008,1,3,0.014219


### На какую сумму, приведенную к валюте 4, каждая из огранизаций выдала займов в июне 2017 года?

In [149]:
import time
import numpy as np
import datetime

Соединим таблицы и originators по столбцу организации, чтобы знать какой валютой пользуется организация, выдавшая займ. Оставим в данной таблице только необходимые нам колонки

In [150]:
mrg = pd.merge(loans, originators, on='originatorId')
mrg = mrg[['originatorId', 'amount', 'issuedDatetime', 'currency']]

Уберем из таблицы записи, которые не относятся к июню 2017-го

In [151]:
june_begin = time.mktime(time.strptime('2017-06-01', '%Y-%m-%d'))
june_end = time.mktime(time.strptime('2017-07-01', '%Y-%m-%d'))
mrg = mrg.drop(index=mrg[mrg.issuedDatetime < june_begin].index)
mrg = mrg.drop(index=mrg[mrg.issuedDatetime >= june_end].index)

Проверим, что у нас нет записей для займов, уже выданных в валюте четыре, так как в таблице перевода валют нет коэффициента для перевода валюты саму в себя и это может вызвать накладку

In [152]:
mrg[mrg.currency == 4].shape[0]

0

Преобразуем дату выдачи займа к виду, в котором хранятся даты в таблице перевода валют

In [153]:
mrg.issuedDatetime = map(lambda x: datetime.datetime.fromtimestamp(x).strftime('%Y%m%d'),
                        mrg.issuedDatetime)

Приведем дату в таблице перевода валют к строковому типу, а также удалим те записи данной таблицы, которые не содержат полезной нам информации про перевод к четвертой валюте

In [154]:
rates.day = map(lambda x: str(x), rates.day)
rates = rates.drop(index=rates[rates.targetCurrencyId != 4].index)

Соединением таблиц получим итоговую таблицу операций в валютах и со столбцом коэффициента перевода в четвертую валюту

In [155]:
operations = pd.merge(mrg, rates, left_on=['issuedDatetime', 'currency'], 
              right_on=['day', 'sourceCurrencyId'])
operations = operations.drop(columns=['sourceCurrencyId', 'targetCurrencyId', 'currency'])
operations.rename(columns={"rate" : "rate_to_4"}, inplace=True)
operations.head()

,originatorId,amount,issuedDatetime,day,rate_to_4
0,2,3205.49,20170601,20170601,0.20853
1,2,4487.69,20170601,20170601,0.20853
2,2,6410.23,20170601,20170601,0.20853
3,2,2500.00,20170601,20170601,0.20853
4,2,1100.00,20170601,20170601,0.20853


Посчитаем сумму каждой операции в четвертой валюте

In [156]:
operations['amount_in_4'] = map(lambda x: x[0] / x[1],  zip(operations.amount, operations.rate_to_4))

Оставим необходимые нам столбцы имени организации и суммы операции в четвертой валюте

In [157]:
operations = operations[['originatorId', 'amount_in_4']]

Группировкой таблицы по номеру организации получим итоговую сумму всех операций для каждой организации

In [183]:
#чтобы выводить в более наглядном формате, не в scientific notation
pd.set_option('float_format', '{:f}'.format) 


grouper = operations.groupby(by=['originatorId'])
grouped = grouper.sum()
grouped

,amount_in_4
originatorId,
2,8172851.288076
6,54956.978577
8,6363707.876944
16,16929295.144457
26,594469.058584
27,94091881.250168
61,15364411.758557
68,18498.786773
72,895995.515245


В соответствии с заданием, которое просит данную информацию для каждой организации учтем так же и те, которые не выдавали займы в июне 2017-го

In [184]:
grouped['originatorId'] = grouped.index
answer = pd.merge(originators, grouped, on='originatorId', how='left')
answer.fillna(0)

,originatorId,currency,amount_in_4
0,1,1,0.000000
1,2,7,8172851.288076
2,6,3,54956.978577
3,8,7,6363707.876944
4,16,8,16929295.144457
5,20,3,0.000000
6,21,3,0.000000
7,26,6,594469.058584
8,27,8,94091881.250168
9,30,2,0.000000
